In [ ]:
import pandas as pd

In [ ]:
complex_samples = [
    {
        "NLQ": "what is the full name and id of the college with the largest number of baseball players?",
        "Gold SQL": "SELECT T1.name_full, T1.college_id FROM college AS T1 JOIN player_college AS T2 ON T1.college_id = T2.college_id GROUP BY T1.college_id ORDER BY count(*) DESC LIMIT 1;"
    },
    {
        "NLQ": "What is average salary of the players in the team named 'Boston Red Stockings' ?",
        "Gold SQL": "SELECT avg(T1.salary) FROM salary AS T1 JOIN team AS T2 ON T1.team_id = T2.team_id_br WHERE T2.name = 'Boston Red Stockings';"
    },
    {
        "NLQ": "What are first and last names of players participating in all star game in 1998?",
        "Gold SQL": "SELECT name_first, name_last FROM player AS T1 JOIN all_star AS T2 ON T1.player_id = T2.player_id WHERE YEAR = 1998;"
    },
    {
        "NLQ": "What are the first name, last name and id of the player with the most all star game experiences? Also list the count.",
        "Gold SQL": "SELECT T1.name_first, T1.name_last, T1.player_id, count(*) FROM player AS T1 JOIN all_star AS T2 ON T1.player_id = T2.player_id GROUP BY T1.player_id ORDER BY count(*) DESC LIMIT 1;"
    },
    {
        "NLQ": "How many players enter hall of fame each year?",
        "Gold SQL": "SELECT yearid, count(*) FROM hall_of_fame GROUP BY yearid;"
    },
    {
        "NLQ": "In 2014, what are the id and rank of the team that has the largest average number of attendance?",
        "Gold SQL": "SELECT T2.team_id, T2.rank FROM home_game AS T1 JOIN team AS T2 ON T1.team_id = T2.team_id WHERE T1.year = 2014 GROUP BY T1.team_id ORDER BY avg(T1.attendance) DESC LIMIT 1;"
    },
    {
        "NLQ": "What are the manager's first name, last name and id who won the most manager award?",
        "Gold SQL": "SELECT T1.name_first, T1.name_last, T2.player_id FROM player AS T1 JOIN manager_award AS T2 ON T1.player_id = T2.player_id GROUP BY T2.player_id ORDER BY count(*) DESC LIMIT 1;"
    },
    {
        "NLQ": "Which 3 players won the most player awards? List their full name and id.",
        "Gold SQL": "SELECT T1.name_first, T1.name_last, T1.player_id FROM player AS T1 JOIN player_award AS T2 ON T1.player_id = T2.player_id GROUP BY T1.player_id ORDER BY count(*) DESC LIMIT 3;"
    },
    {
        "NLQ": "List three countries which are the origins of the least players.",
        "Gold SQL": "SELECT birth_country FROM player GROUP BY birth_country ORDER BY count(*) ASC LIMIT 3;"
    },
    {
        "NLQ": "Find all the players' first name and last name who have empty death record.",
        "Gold SQL": "SELECT name_first, name_last FROM player WHERE death_year = '';"
    },
    {
        "NLQ": "What is the average height of the players from the college named 'Yale University'?",
        "Gold SQL": "SELECT avg(T1.height) FROM player AS T1 JOIN player_college AS T2 ON T1.player_id = T2.player_id JOIN college AS T3 ON T3.college_id = T2.college_id WHERE T3.name_full = 'Yale University';"
    },
    {
        "NLQ": "What is the highest salary among each team? List the team name, id and maximum salary.",
        "Gold SQL": "SELECT T1.name, T1.team_id, max(T2.salary) FROM team AS T1 JOIN salary AS T2 ON T1.team_id = T2.team_id GROUP BY T1.team_id;"
    },
    {
        "NLQ": "What are the name and id of the team offering the lowest average salary?",
        "Gold SQL": "SELECT T1.name, T1.team_id FROM team AS T1 JOIN salary AS T2 ON T1.team_id = T2.team_id GROUP BY T1.team_id ORDER BY avg(T2.salary) ASC LIMIT 1;"
    },
    {
        "NLQ": "Find the players' first name and last name who won award both in 1960 and in 1961.",
        "Gold SQL": "SELECT T1.name_first, T1.name_last FROM player AS T1 JOIN player_award AS T2 WHERE T2.year = 1960 INTERSECT SELECT T1.name_first, T1.name_last FROM player AS T1 JOIN player_award AS T2 WHERE T2.year = 1961;"
    },
    {
        "NLQ": "List players' first name and last name who have weight greater than 220 or height shorter than 75.",
        "Gold SQL": "SELECT name_first, name_last FROM player WHERE weight > 220 OR height < 75;"
    },
    {
        "NLQ": "What are the maximum scores the team Boston Red Stockings got when the team won in postseason?",
        "Gold SQL": "SELECT max(T1.wins) FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_winner = T2.team_id_br WHERE T2.name = 'Boston Red Stockings';"
    },
    {
        "NLQ": "How many times did Boston Red Stockings lose in 2009 postseason?",
        "Gold SQL": "SELECT count(*) FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_loser = T2.team_id_br WHERE T2.name = 'Boston Red Stockings' AND T1.year = 2009;"
    },
    {
        "NLQ": "What are the name and id of the team with the most victories in 2008 postseason?",
        "Gold SQL": "SELECT T2.name, T1.team_id_winner FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_winner = T2.team_id_br WHERE T1.year = 2008 GROUP BY T1.team_id_winner ORDER BY count(*) DESC LIMIT 1;"
    },
    {
        "NLQ": "What is the total number of postseason games that team Boston Red Stockings participated in?",
        "Gold SQL": "SELECT count(*) FROM ( SELECT * FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_winner = T2.team_id_br WHERE T2.name = 'Boston Red Stockings' UNION SELECT * FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_loser = T2.team_id_br WHERE T2.name = 'Boston Red Stockings' );"
    },
    {
        "NLQ": "What is the total salary paid by team Boston Red Stockings in 2010?",
        "Gold SQL": "SELECT sum(T1.salary) FROM salary AS T1 JOIN team AS T2 ON T1.team_id = T2.team_id_br WHERE T2.name = 'Boston Red Stockings' AND T1.year = 2010;"
    }
]


In [ ]:
# 1. Imports
import networkx as nx
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 2. Model & Tokenizer Path
model_path = "seeklhy/OmniSQL-7B"

# 3. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 4. Load Model (with smart dtype/device)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# 5. Model in evaluation mode (recommended for inference)
model.eval()

# 6. Utility: Move to CUDA if available (optional; device_map usually handles this)
if torch.cuda.is_available():
    model = model.to("cuda")
else:
    model = model.to("cpu")


In [ ]:
!pip install networkx

In [ ]:
schema_dict = {
    "player": {
        "description": "Baseball player; biographical info and unique player_id.",
        "synonyms": ["athlete", "baseball player", "players"],
        "columns": {
            "player_id": {
                "type": "TEXT (PK)",
                "synonyms": ["id", "player identifier"],
                "example": "willite01"
            },
            "birth_year": {
                "type": "INTEGER",
                "synonyms": ["year of birth", "born"],
                "example": 1918
            },
            "birth_country": {
                "type": "TEXT",
                "synonyms": ["country of birth"],
                "example": "USA"
            },
            "name_first": {
                "type": "TEXT",
                "synonyms": ["first name", "given name"],
                "example": "Ted"
            },
            "name_last": {
                "type": "TEXT",
                "synonyms": ["last name", "surname", "family name"],
                "example": "Williams"
            },
            "debut": {
                "type": "DATE",
                "synonyms": ["debut date", "first game"],
                "example": "1939-04-20"
            },
            "final_game": {
                "type": "DATE",
                "synonyms": ["last game", "retirement date"],
                "example": "1960-09-28"
            },
            "bats": {
                "type": "TEXT",
                "synonyms": ["batting hand", "bat side"],
                "example": "L"
            },
            "throws": {
                "type": "TEXT",
                "synonyms": ["throwing hand", "throw side"],
                "example": "R"
            },
            "weight": {
                "type": "INTEGER",
                "synonyms": ["weight (lbs)"],
                "example": 205
            },
            "height": {
                "type": "INTEGER",
                "synonyms": ["height (inches)"],
                "example": 74
            }
        }
    },
    "team": {
        "description": "Baseball team; franchise info.",
        "synonyms": ["club", "baseball team", "teams"],
        "columns": {
            "team_id": {
                "type": "TEXT (PK)",
                "synonyms": ["team identifier", "id"],
                "example": "BOS"
            },
            "team_id_br": {
                "type": "TEXT",
                "synonyms": ["baseball reference id"],
                "example": "BOS"
            },
            "name": {
                "type": "TEXT",
                "synonyms": ["team name", "franchise"],
                "example": "Red Sox"
            },
            "city": {
                "type": "TEXT",
                "synonyms": ["home city", "city name"],
                "example": "Boston"
            },
            "league": {
                "type": "TEXT",
                "synonyms": ["league", "division"],
                "example": "AL"
            },
            "year": {
                "type": "INTEGER",
                "synonyms": ["season", "year"],
                "example": 2004
            },
            "attendance": {
                "type": "INTEGER",
                "synonyms": ["game attendance", "fans attended"],
                "example": 2500000
            },
            "rank": {
                "type": "INTEGER",
                "synonyms": ["standing", "position"],
                "example": 1
            }
        }
    },
    "player_award": {
        "description": "Awards received by players.",
        "synonyms": ["award", "player award"],
        "columns": {
            "award_id": {"type": "TEXT (PK)", "example": "MVP"},
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "year": {"type": "INTEGER", "example": 1946}
        }
    },
    "hall_of_fame": {
        "description": "Hall of Fame inductions.",
        "synonyms": ["hof", "hall of fame"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "yearid": {"type": "INTEGER", "synonyms": ["year", "induction year"], "example": 1966},
            "inducted": {"type": "TEXT", "synonyms": ["inducted status", "inducted (Y/N)"], "example": "Y"}
        }
    },
    "salary": {
        "description": "Player salary per year and team.",
        "synonyms": ["salaries", "player salary"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "team_id": {"type": "TEXT (FK)", "example": "BOS"},
            "year": {"type": "INTEGER", "example": 1946},
            "salary": {"type": "INTEGER", "synonyms": ["player salary", "compensation"], "example": 75000}
        }
    },
    "appearances": {
        "description": "Player appearances by position for a season.",
        "synonyms": ["season appearances"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "team_id": {"type": "TEXT (FK)", "example": "BOS"},
            "year": {"type": "INTEGER", "example": 1946},
            "g_c": {"type": "INTEGER", "synonyms": ["games as catcher"], "example": 0},
            "g_p": {"type": "INTEGER", "synonyms": ["games as pitcher"], "example": 0},
            "g_1b": {"type": "INTEGER", "synonyms": ["games at first base"], "example": 0},
            "g_2b": {"type": "INTEGER", "synonyms": ["games at second base"], "example": 0},
            "g_3b": {"type": "INTEGER", "synonyms": ["games at third base"], "example": 0},
            "g_ss": {"type": "INTEGER", "synonyms": ["games at shortstop"], "example": 0},
            "g_lf": {"type": "INTEGER", "synonyms": ["games in left field"], "example": 140},
            "g_cf": {"type": "INTEGER", "synonyms": ["games in center field"], "example": 0},
            "g_rf": {"type": "INTEGER", "synonyms": ["games in right field"], "example": 0},
            "g_dh": {"type": "INTEGER", "synonyms": ["games as designated hitter"], "example": 0}
        }
    },
    "batting": {
        "description": "Player batting statistics per season.",
        "synonyms": ["batting stats", "player batting"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "team_id": {"type": "TEXT (FK)", "example": "BOS"},
            "year": {"type": "INTEGER", "example": 1946},
            "games": {"type": "INTEGER", "synonyms": ["games played"], "example": 150},
            "at_bats": {"type": "INTEGER", "synonyms": ["at bats"], "example": 520},
            "hits": {"type": "INTEGER", "synonyms": ["base hits"], "example": 180},
            "hr": {"type": "INTEGER", "synonyms": ["home runs"], "example": 38},
            "rbi": {"type": "INTEGER", "synonyms": ["runs batted in"], "example": 123}
        }
    },
    "batting_postseason": {
        "description": "Player batting statistics in postseason.",
        "synonyms": ["postseason batting", "playoff batting"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "team_id": {"type": "TEXT (FK)", "example": "BOS"},
            "year": {"type": "INTEGER", "example": 1946},
            "games": {"type": "INTEGER", "example": 7},
            "at_bats": {"type": "INTEGER", "example": 24},
            "hits": {"type": "INTEGER", "example": 5},
            "hr": {"type": "INTEGER", "example": 0},
            "rbi": {"type": "INTEGER", "example": 3}
        }
    },
    "player_college": {
        "description": "Mapping between players and colleges attended.",
        "synonyms": ["college attendance", "player college"],
        "columns": {
            "player_id": {"type": "TEXT (FK)", "example": "willite01"},
            "college_id": {"type": "TEXT (FK)", "example": "texas"}
        }
    },
    "college": {
        "description": "Colleges attended by players.",
        "synonyms": ["university", "college", "colleges"],
        "columns": {
            "college_id": {"type": "TEXT (PK)", "example": "texas"},
            "name_full": {"type": "TEXT", "synonyms": ["college name", "university name"], "example": "University of Texas at Austin"}
        }
    },
    "home_game": {
        "description": "Home game details: attendance, park, year, team.",
        "synonyms": ["home games", "hosted game"],
        "columns": {
            "team_id": {"type": "TEXT (FK)", "example": "BOS"},
            "park_id": {"type": "TEXT (FK)", "example": "bosparka"},
            "year": {"type": "INTEGER", "example": 2004},
            "attendance": {"type": "INTEGER", "example": 35000},
            "city": {"type": "TEXT", "example": "Boston"},
            "games": {"type": "INTEGER", "synonyms": ["games played"], "example": 81}
        }
    },
    "park": {
        "description": "Baseball parks.",
        "synonyms": ["ballpark", "stadium", "parks"],
        "columns": {
            "park_id": {"type": "TEXT (PK)", "example": "bosparka"},
            "park_name": {"type": "TEXT", "synonyms": ["name", "stadium name"], "example": "Fenway Park"},
            "state": {"type": "TEXT", "example": "MA"},
            "city": {"type": "TEXT", "example": "Boston"}
        }
    },
    "postseason": {
        "description": "Postseason results. Winning/losing teams, year, round.",
        "synonyms": ["playoffs", "postseason series"],
        "columns": {
            "year": {"type": "INTEGER", "example": 2004},
            "round": {"type": "TEXT", "synonyms": ["playoff round"], "example": "WS"},
            "team_id_winner": {"type": "TEXT (FK)", "example": "BOS"},
            "team_id_loser": {"type": "TEXT (FK)", "example": "STL"},
            "league_id_winner": {"type": "TEXT", "example": "AL"},
            "wins": {"type": "INTEGER", "example": 4},
            "ties": {"type": "INTEGER", "example": 0}
        }
    },
    "team_franchise": {
        "description": "Team franchise metadata.",
        "synonyms": ["franchise", "team franchise"],
        "columns": {
            "franchise_id": {"type": "TEXT (PK)", "example": "BOS"},
            "active": {"type": "TEXT", "synonyms": ["active status"], "example": "Y"}
        }
    }
}

# Relationships with narrative, cardinality, and join columns
schema_relationships = {
    ("player", "player_award"): {
        "columns": ("player.player_id", "player_award.player_id"),
        "cardinality": "one-to-many",
        "description": "Each player can receive multiple awards; link via player_id."
    },
    ("player", "hall_of_fame"): {
        "columns": ("player.player_id", "hall_of_fame.player_id"),
        "cardinality": "one-to-one or zero",
        "description": "A player may be inducted into the Hall of Fame; link via player_id."
    },
    ("player", "salary"): {
        "columns": ("player.player_id", "salary.player_id"),
        "cardinality": "one-to-many",
        "description": "A player may have multiple salary records (one per year/team)."
    },
    ("player", "appearances"): {
        "columns": ("player.player_id", "appearances.player_id"),
        "cardinality": "one-to-many",
        "description": "A player can have multiple appearance records by season/team."
    },
    ("player", "batting"): {
        "columns": ("player.player_id", "batting.player_id"),
        "cardinality": "one-to-many",
        "description": "A player can have multiple batting records (per season/team)."
    },
    ("player", "batting_postseason"): {
        "columns": ("player.player_id", "batting_postseason.player_id"),
        "cardinality": "one-to-many",
        "description": "A player can have multiple postseason batting records."
    },
    ("player", "player_college"): {
        "columns": ("player.player_id", "player_college.player_id"),
        "cardinality": "one-to-many",
        "description": "A player can attend multiple colleges."
    },
    ("college", "player_college"): {
        "columns": ("college.college_id", "player_college.college_id"),
        "cardinality": "one-to-many",
        "description": "Each college can have many players."
    },
    ("team", "appearances"): {
        "columns": ("team.team_id", "appearances.team_id"),
        "cardinality": "one-to-many",
        "description": "Each team has multiple player appearance records per season."
    },
    ("team", "batting"): {
        "columns": ("team.team_id", "batting.team_id"),
        "cardinality": "one-to-many",
        "description": "Each team has multiple player batting records per season."
    },
    ("team", "salary"): {
        "columns": ("team.team_id", "salary.team_id"),
        "cardinality": "one-to-many",
        "description": "Each team pays salaries to multiple players."
    },
    ("team", "home_game"): {
        "columns": ("team.team_id", "home_game.team_id"),
        "cardinality": "one-to-many",
        "description": "A team hosts many home games."
    },
    ("team", "postseason"): {
        "columns": ("team.team_id", "postseason.team_id_winner"),
        "cardinality": "one-to-many",
        "description": "A team can be the winner in multiple postseason series."
    },
    ("home_game", "park"): {
        "columns": ("home_game.park_id", "park.park_id"),
        "cardinality": "many-to-one",
        "description": "Each home game is played at a park."
    },
    ("salary", "team"): {
        "columns": ("salary.team_id", "team.team_id"),
        "cardinality": "many-to-one",
        "description": "Each salary record refers to the paying team."
    },
    ("batting_postseason", "team"): {
        "columns": ("batting_postseason.team_id", "team.team_id"),
        "cardinality": "many-to-one",
        "description": "Each postseason batting record links to a team."
    },
    ("postseason", "team_winner"): {
        "columns": ("postseason.team_id_winner", "team.team_id"),
        "cardinality": "many-to-one",
        "description": "Each postseason series has a winner (team)."
    },
    ("postseason", "team_loser"): {
        "columns": ("postseason.team_id_loser", "team.team_id"),
        "cardinality": "many-to-one",
        "description": "Each postseason series has a loser (team)."
    },
    ("team", "team_franchise"): {
        "columns": ("team.team_id", "team_franchise.franchise_id"),
        "cardinality": "many-to-one",
        "description": "Each team belongs to a franchise."
    }
}


In [ ]:
import networkx as nx

def build_schema_graph(schema_dict, schema_relationships):
    G = nx.MultiDiGraph()
    # Tables & synonyms
    for table, tmeta in schema_dict.items():
        G.add_node(table, type="table", description=tmeta['description'])
        for syn in tmeta.get('synonyms', []):
            G.add_node(f"{table}_syn_{syn}", type="synonym", value=syn)
            G.add_edge(table, f"{table}_syn_{syn}", type="synonym")
        for col, cmeta in tmeta['columns'].items():
            col_node = f"{table}.{col}"
            G.add_node(col_node, type="column", col_type=cmeta['type'])
            G.add_edge(table, col_node, type="has_column")
            for syn in cmeta.get('synonyms', []):
                syn_node = f"{col_node}_syn_{syn}"
                G.add_node(syn_node, type="synonym", value=syn)
                G.add_edge(col_node, syn_node, type="synonym")
            if 'example' in cmeta:
                ex_node = f"{col_node}_example"
                G.add_node(ex_node, type="example", value=cmeta['example'])
                G.add_edge(col_node, ex_node, type="example")
    # Relationships
    for (t1, t2), rel in schema_relationships.items():
        rel_node = f"{t1}__{t2}_rel"
        G.add_node(rel_node, type="relationship", description=rel['description'], cardinality=rel['cardinality'])
        G.add_edge(t1, rel_node, type="relationship")
        G.add_edge(rel_node, t2, type="relationship")
        G.add_edge(rel_node, f"{t1}.{rel['columns'][0].split('.')[1]}", type="join_column")
        G.add_edge(rel_node, f"{t2}.{rel['columns'][1].split('.')[1]}", type="join_column")
    return G

G = build_schema_graph(schema_dict, schema_relationships)


In [ ]:
def get_relevant_schema_from_kg(nlq, kg, schema_dict, schema_relationships, max_hops=2):
    nlq_lc = nlq.lower()
    table_hits, column_hits = set(), set()

    # 1. Match table and column names (and synonyms) in NLQ
    for node, data in kg.nodes(data=True):
        node_type = data.get('type')
        if node_type == 'table' and node.lower() in nlq_lc:
            table_hits.add(node)
        if node_type == 'column' and node.lower().split('.')[-1] in nlq_lc:
            column_hits.add(node)
        if node_type == 'synonym' and data.get('value','').lower() in nlq_lc:
            for pred in kg.predecessors(node):
                if kg.nodes[pred].get('type') == 'table':
                    table_hits.add(pred)
                elif kg.nodes[pred].get('type') == 'column':
                    column_hits.add(pred)

    # 2. For each table/column hit, get neighbors within max_hops for full context
    relevant_tables, relevant_columns, relevant_rels = set(), set(), set()
    for node in list(table_hits) + list(column_hits):
        for n, d in nx.single_source_shortest_path_length(kg, node, cutoff=max_hops).items():
            ntype = kg.nodes[n].get('type')
            if ntype == 'table':
                relevant_tables.add(n)
            elif ntype == 'column':
                relevant_columns.add(n)
            elif ntype == 'relationship':
                relevant_rels.add(n)

    # 3. Build reduced schema_dict and relationships_dict for prompt
    sub_schema = {}
    for t in relevant_tables:
        if t in schema_dict:
            cols = {}
            for col, meta in schema_dict[t]['columns'].items():
                if f"{t}.{col}" in relevant_columns:
                    cols[col] = meta
            sub_schema[t] = {**schema_dict[t], "columns": cols if cols else schema_dict[t]['columns']}
    sub_rels = {}
    for rel in relevant_rels:
        parts = rel.split("__")
        if len(parts) == 2:
            t1, t2_card = parts
            t2 = t2_card.replace("_rel", "")
            key = (t1, t2)
            if key in schema_relationships:
                sub_rels[key] = schema_relationships[key]
    return sub_schema, sub_rels


In [ ]:
def build_single_shot_prompt(nlq, example_nlq, example_sql, schema_dict, relationships_dict):
    def render_table(table, meta):
        s = f"Table: {table}\n  Description: {meta['description']}\n"
        s += "  Columns:\n"
        for col, cmeta in meta['columns'].items():
            s += f"    - {col} ({cmeta['type']})"
            if 'synonyms' in cmeta:
                s += f" [synonyms: {', '.join(cmeta['synonyms'])}]"
            s += "\n"
        return s

    schema_text = "\n".join([render_table(t, schema_dict[t]) for t in schema_dict])

    rel_text = ""
    if relationships_dict:
        rel_text = "Relationships:\n"
        for (t1, t2), rmeta in relationships_dict.items():
            rel_text += f"  - {t1} <-> {t2}: {rmeta['description']} (Join: {rmeta['columns'][0]} = {rmeta['columns'][1]}; Cardinality: {rmeta['cardinality']})\n"

    example = f"""Example:
NLQ: {example_nlq}
SQL: {example_sql}
"""

    prompt = f"""
You are an expert SQL agent for the Spider Baseball database.
OUTPUT ONLY THE SQL QUERY—NO explanations, markdown, or commentary. Do not use SQL comments.

Schema:
{schema_text}

{rel_text}

{example}
Now generate ONLY the SQL for this NLQ (no explanation, no markdown):

NLQ: {nlq}
SQL:
""".strip()

    return prompt


In [ ]:
# For a single NLQ:
nlq = complex_samples[0]["NLQ"]
example_nlq = complex_samples[1]["NLQ"]
example_sql = complex_samples[1]["Gold SQL"]

# Get minimized schema and rels for this NLQ:
sub_schema, sub_rels = get_relevant_schema_from_kg(nlq, G)

# Build the prompt:
prompt = build_single_shot_prompt(
    nlq=nlq,
    example_nlq=example_nlq,
    example_sql=example_sql,
    schema_dict=sub_schema,
    relationships_dict=sub_rels
)

print(prompt)


In [ ]:
def infer_sql(prompt, tokenizer, model, max_tokens=256):
    """
    Given a prompt, uses the model and tokenizer to generate SQL query.
    Returns the decoded SQL (not cleaned—do that after if needed).
    """
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            do_sample=False,      # Greedy decoding
            eos_token_id=tokenizer.eos_token_id
        )
    # Only decode the new tokens (generated part)
    generated = outputs[0][input_ids.shape[-1]:]
    result_sql = tokenizer.decode(generated, skip_special_tokens=True).strip()
    return result_sql


In [ ]:
def batch_generate_sql(
    samples,
    example_nlq,
    example_sql,
    schema_dict,
    relationships_dict,
    graph,               # The query graph (networkx.MultiDiGraph)
    tokenizer,
    model,
    max_tokens=256,
    minimize_schema=True # If True, use KG to shrink prompt
):
    results = []
    for i, sample in enumerate(samples):
        nlq = sample["NLQ"]

        # Option 1: Use KG to minimize prompt schema (recommended for large schemas)
        if minimize_schema:
            sub_schema, sub_rels = get_relevant_schema_from_kg(
                nlq, graph, schema_dict, relationships_dict
            )
        else:
            sub_schema, sub_rels = schema_dict, relationships_dict

        # Build prompt
        prompt = build_single_shot_prompt(
            nlq=nlq,
            example_nlq=example_nlq,
            example_sql=example_sql,
            schema_dict=sub_schema,
            relationships_dict=sub_rels
        )
        result_sql = infer_sql(
            prompt,
            tokenizer=tokenizer,
            model=model,
            max_tokens=max_tokens
        )
        results.append({
            "NLQ": sample["NLQ"],
            "Gold SQL": sample["Gold SQL"],
            "Predicted SQL": result_sql
        })
        print(f"\n--- Query {i+1} ---")
        print("Question:", sample['NLQ'])
        print("Gold SQL:\n", sample['Gold SQL'])
        print("Generated SQL:\n", result_sql)
    return results


In [ ]:
example_nlq = complex_samples[0]['NLQ']
example_sql = complex_samples[0]['Gold SQL']

results = batch_generate_sql(
    samples=complex_samples,
    example_nlq=example_nlq,
    example_sql=example_sql,
    schema_dict=schema_dict,
    relationships_dict=schema_relationships,
    graph=G,
    tokenizer=tokenizer,
    model=model,
    max_tokens=256,
    minimize_schema=True
)


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select your baseball.sqlite file here

In [ ]:
import sqlite3
# Path to your database (update this)
db_path = 'baseball_1.sqlite'  # <--- UPDATE THIS PATH!
conn = sqlite3.connect(db_path)

In [ ]:
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

In [ ]:
import re

def clean_predicted_sql(pred_sql: str) -> str:
    """
    Cleans predicted SQL query to remove markdown, explanations, and only keep first statement.
    Keeps only the first complete SQL statement (ending with a semicolon).
    """
    if not isinstance(pred_sql, str):
        return ""
    # Remove markdown code blocks and backticks (works for ```sql, ``` and `)
    cleaned = re.sub(r"^```(?:sql)?|```|`", "", pred_sql, flags=re.MULTILINE | re.IGNORECASE)
    # Remove explanations before the first SQL verb (case-insensitive, matches on word boundary)
    cleaned_split = re.split(r'(?i)\b(SELECT|INSERT|UPDATE|DELETE)\b', cleaned, maxsplit=1)
    if len(cleaned_split) > 2:
        cleaned = cleaned_split[1] + cleaned_split[2]
    elif len(cleaned_split) == 2:
        cleaned = cleaned_split[1]
    else:
        cleaned = cleaned_split[0]
    # Only keep up to the first semicolon (inclusive, in case of multiple statements)
    semicolon_pos = cleaned.find(';')
    if semicolon_pos != -1:
        cleaned = cleaned[:semicolon_pos + 1]
    # Remove trailing/leading whitespace and markdown artifacts
    cleaned = cleaned.strip()
    cleaned = re.sub(r"```", "", cleaned)
    cleaned = re.sub(r"[`*#\-]+", "", cleaned)
    # Ensure a single semicolon at the end
    cleaned = cleaned.rstrip(';') + ';'
    # Collapse internal whitespace to single space
    cleaned = re.sub(r"\s+", " ", cleaned)
    return cleaned.strip()


In [ ]:
import time

def run_query(sql, conn):
    """
    Execute a SQL query and return canonicalized results for comparison.

    Args:
        sql (str): The SQL statement to execute.
        conn (sqlite3.Connection): The SQLite database connection.

    Returns:
        result (tuple of tuples): Sorted, hashable result rows.
        error (str or None): Error message if any exception occurs, else None.
        elapsed (float): Time taken to execute the query, in seconds.
    """
    try:
        start = time.time()
        cur = conn.execute(sql)
        rows = cur.fetchall()
        elapsed = time.time() - start
        # Canonicalize result for consistent comparison (sorted, hashable)
        result = tuple(sorted(tuple(row) for row in rows))
        return result, None, elapsed
    except Exception as e:
        return None, str(e), 0.0


In [ ]:
import time
results_df = pd.DataFrame(results)

exec_results = []

for i, row in results_df.iterrows():
    nlq = row["NLQ"]
    gold_sql = row["Gold SQL"]
    raw_predicted_sql = row["Predicted SQL"]

    # Clean model output to get valid SQL
    pred_sql = clean_predicted_sql(raw_predicted_sql)

    # Run gold SQL
    gold_result, gold_error, gold_time = run_query(gold_sql, conn)

    # Run predicted SQL
    pred_result, pred_error, pred_time = run_query(pred_sql, conn)

    # Execution accuracy: results must match and no errors in either
    exec_match = (
        (gold_result == pred_result) and
        (gold_error is None) and
        (pred_error is None)
    )

    exec_results.append({
        "NLQ": nlq,
        "Gold SQL": gold_sql,
        "Predicted SQL": pred_sql,
        "Gold Result": gold_result,
        "Predicted Result": pred_result,
        "Gold Error": gold_error,
        "Pred Error": pred_error,
        "Gold Time (s)": gold_time,
        "Pred Time (s)": pred_time,
        "Exec_Match": exec_match
    })

# If you want to convert this to a DataFrame at the end:
import pandas as pd
exec_results_df = pd.DataFrame(exec_results)


In [ ]:
exec_df = pd.DataFrame(exec_results)
accuracy = exec_df["Exec_Match"].mean()

print(f"Execution Accuracy: {accuracy:.2%} ({exec_df.Exec_Match.sum()} / {len(exec_df)})")


In [ ]:
# Only in Jupyter/Colab:
from IPython.display import display

print("\nMismatched Results:")
display(exec_df[~exec_df["Exec_Match"]][[
    "NLQ", "Gold SQL", "Predicted SQL", "Gold Result", "Predicted Result", "Gold Error", "Pred Error"
]])